In [1]:
import time
import os
import numba

from coffea.nanoevents import BaseSchema
 
import awkward as ak
import numpy as np
from coffea import processor, hist

from coffea.lookup_tools import extractor
from coffea.nanoevents.methods import candidate
ak.behavior.update(candidate.behavior)

import mplhep
import matplotlib.pyplot as plt
plt.style.use(mplhep.style.CMS)

In [18]:
muon_cols = ['Muon_charge', 'Muon_dxy', 'Muon_dxyErr', 'Muon_dz', 'Muon_dzErr', 'Muon_eta', 'Muon_isGlobal', 'Muon_mass',
             'Muon_phi', 'Muon_pt', 'Muon_ptErr', 'Muon_softId', 'Muon_vtxIdx', 'Muon_vtxFlag', 'Muon_simIdx']

dimu_cols = ['Dimu_pt', 'Dimu_eta', 'Dimu_phi', 'Dimu_rap', 'Dimu_mass', 'Dimu_charge', 'Dimu_vtxIdx', 'Dimu_chi2', 'Dimu_dl',
             'Dimu_dlErr', 'Dimu_dlSig', 'Dimu_cosphi', 'Dimu_x', 'Dimu_y', 'Dimu_z', 'Dimu_t1muIdx', 'Dimu_t2muIdx',]

d0_cols = ['D0_pt', 'D0_eta', 'D0_phi', 'D0_rap', 'D0_mass12', 'D0_mass21', 'D0_vtxIdx', 'D0_chi2', 'D0_dl', 'D0_dlErr', 'D0_dlSig',
           'D0_cosphi', 'D0_x', 'D0_y', 'D0_z', 'D0_hasMuon',
           'D0_t1pt', 'D0_t1eta', 'D0_t1phi', 'D0_t1chindof', 'D0_t1nValid', 'D0_t1nPix', 'D0_t1dxy', 'D0_t1dz', 'D0_t1chg', 
           'D0_t2pt', 'D0_t2eta', 'D0_t2phi', 'D0_t2chindof', 'D0_t2nValid', 'D0_t2nPix', 'D0_t2dxy', 'D0_t2dz', 'D0_t2chg',]

dstar_cols = ['Dstar_pt', 'Dstar_eta', 'Dstar_phi', 'Dstar_rap', 'Dstar_deltam', 'Dstar_deltamr', 'Dstar_vtxIdx', 'Dstar_hasMuon',
              'Dstar_D0pt', 'Dstar_D0eta', 'Dstar_D0phi', 'Dstar_D0mass', 'Dstar_D0chi2', 'Dstar_D0dl', 'Dstar_D0dlErr',
              'Dstar_D0dlSig', 'Dstar_D0cosphi', 'Dstar_D0x', 'Dstar_D0y', 'Dstar_D0z',
              'Dstar_Kpt', 'Dstar_Keta', 'Dstar_Kphi', 'Dstar_KvtxIdx', 'Dstar_Kchindof', 'Dstar_KnValid', 'Dstar_KnPix', 'Dstar_Kdxy',
              'Dstar_Kdz', 'Dstar_Kchg',
              'Dstar_pipt', 'Dstar_pieta', 'Dstar_piphi', 'Dstar_pivtxIdx', 'Dstar_pichindof', 'Dstar_pinValid', 'Dstar_pinPix',
              'Dstar_pidxy', 'Dstar_pidz', 'Dstar_pichg',
              'Dstar_pispt', 'Dstar_piseta', 'Dstar_pisphi', 'Dstar_pisvtxIdx', 'Dstar_pischindof', 'Dstar_pisnValid', 'Dstar_pisnPix',
              'Dstar_pisdxy', 'Dstar_pisdz', 'Dstar_simIdx', 'Dstar_D0simIdx']

gen_part_cols = ['GenPart_eta', 'GenPart_genPartIdxMother', 'GenPart_mass', 'GenPart_pdgId', "GenPart_phi", "GenPart_pt", 'GenPart_status',
                'GenPart_Id', 'GenPart_parpdgId', 'GenPart_sparpdgId', 'GenPart_numberOfDaughters', 'GenPart_nstchgdaug', 'GenPart_vx', 
                'GenPart_vy', 'GenPart_vz', 'GenPart_mvx', 'GenPart_mvy', 'GenPart_mvz', 'GenPart_recIdx']

primary_vertex_aod_cols = ['nPVtx', 'PVtx_Id', 'PVtx_isGood', 'PVtx_x', 'PVtx_y', 'PVtx_z', 'PVtx_ntrk', 'PVtx_x', 'PVtx_sumPt']

D0_PDG_MASS = 1.864

def get_vars_dict(events, col_list):
    dict = {}
    col = ''
    for c in col_list:
        if c.startswith('Muon'):
            col = c[5:]
        elif c.startswith('Dimu'):
            col = c[4:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('D0'):
            col = c[2:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('Dstar'):
            col = c[5:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('PVtx'):
            col = c[5:]
        elif c.startswith("GenPart"):
            col = c[8:]
        else:
            Exception('Not good!')

        if col == 'x' or col == 'y' or col == 'z':
            col = 'vtx_' + col

        if len(events[c]) == 0:
            dict[col] = np.array([])
        else:
            dict[col] = events[c]
    return dict

In [3]:
import re

files = []
with os.scandir("/eos/user/m/mabarros/Monte_Carlo/2017/0000") as it:
    for file in it:
        if file.name.endswith('.root') and (file.stat().st_size != 0):
            files.append(file.path)

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]
            
files.sort(key=natural_keys)

In [4]:
def create_plot1d(hist1d, log=False, density=False, ax=None):
    from matplotlib.offsetbox import AnchoredOffsetbox, TextArea
    plt.style.use(mplhep.style.CMS)
    plt.rcParams.update({
        'font.size': 16,
        'axes.titlesize': 18,
        'axes.labelsize': 18,
        'xtick.labelsize': 14,
        'ytick.labelsize': 14
    })
    fill_opts = {
    'alpha': 0.8,
    'edgecolor':(0,0,0,.5)
    }
    
    ax = hist.plot1d(hist1d, ax=ax, fill_opts=fill_opts, density=density)
    
    if log:
        ax.set_yscale('log')
        ax.set_ylim(1, None)
    else:
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,3), useMathText=True)
    
    axis = hist1d.axes()[0]
    centers = axis.centers()
    values = np.where(hist1d.values().get(()) < 0, 0, hist1d.values().get(()))
    
    # compute mean and std:
    mean = np.sum(values*centers)/np.sum(values)
    std = np.sqrt(np.sum(values*((centers - mean)**2))/np.sum(values))
    
    annotation = TextArea(f"Total: {np.sum(values):.2e}" \
                    + "\n" + f"Mean: {mean:.2e}" \
                    + "\n" + f"Std: {std:.2e}", textprops=dict(size=14))
    
    at = AnchoredOffsetbox('upper right', child=annotation)
    at.patch.set_facecolor('None')
    #ax.add_artist(at)
    
    ax.legend().remove()
    
    return ax

In [17]:
pileup_file = "pileup_weight.root"
def extract_mc_weight(pileup_file):
    # creates the extractor
    ext = extractor()
    # add the weights from the pileup histograms
    ext.add_weight_sets(["weight_histogram weight_histogram " + pileup_file])
    ext.finalize()

    evaluator = ext.make_evaluator()

    return evaluator

In [47]:
class ReweightTestProcessor(processor.ProcessorABC):
    def __init__(self):
        self._accumulator = processor.dict_accumulator({
            'cutflow': processor.defaultdict_accumulator(int),
            'n_dimu': processor.value_accumulator(int),
            'n_muon': processor.value_accumulator(int),
            'n_dstar': processor.value_accumulator(int),
            'dimu_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,J/\Psi}$ [GeV]", 40, 0, 100)),
            'dimu_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{J/\Psi}$", 60, -4, 4)),
            'dimu_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{J/\Psi}[rad]$", 60, -3.5, 3.5)),
            'dimu_mass': hist.Hist("Events", hist.Bin("mass", r"$mass_{J/\Psi} [GeV]$", 100, 2.95, 3.25)),
            'muon_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,\mu}$ matched [GeV]", 50, 0, 25)),
            'muon_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{\mu} matched$", 60, -10, 10)),
            'muon_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{\mu} matched [rad]$", 60, -3.5, 3.5)),
            'dstar_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,D*} matched$", 40, 0, 20)),
            'dstar_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{D*} matched$", 60, -4, 4)),
            'dstar_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{D*}$ matched", 60, -3.5, 3.5)),
            'dstar_deltamr': hist.Hist("Events", hist.Bin("deltamr", "$\Delta m_{D*}$ matched", 50, 0.138, 0.162)),
        })

    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, events):
        output = self.accumulator.identity()
        
        # test if there is any events in the file
        if len(events) == 0:
            return output
        
        # Collection extraction
        Dimus = ak.zip({**get_vars_dict(events, dimu_cols)}, with_name="PtEtaPhiMCandidate")
        Muons = ak.zip({**get_vars_dict(events, muon_cols)}, with_name="PtEtaPhiMCandidate")
        Dstars = ak.zip({'mass': (events.Dstar_D0mass + events.Dstar_deltamr),
                        'charge': events.Dstar_pischg,
                        **get_vars_dict(events, dstar_cols)}, 
                        with_name="PtEtaPhiMCandidate")
        GenPart = ak.zip({**get_vars_dict(events, gen_part_cols)}, with_name="PtEtaPhiMCandidate")
        PrimVertex = ak.zip({**get_vars_dict(events, primary_vertex_aod_cols)})
        nPVtx = events.nPVtx
        
        output['cutflow']['Number of events'] += len(events)
        output['cutflow']['Number of Dimu'] += ak.sum(ak.num(Dimus))
        output['cutflow']['Number of Muons'] += ak.sum(ak.num(Muons))

        ## Rec dimuon cuts
        
        #Dimu cuts charge = 0, mass cuts and chi2...
        Dimu = Dimus[Dimus.charge == 0]
        output['cutflow']['Dimu 0 charge'] += ak.sum(ak.num(Dimu))

        Dimu = Dimu[(Dimu.mass > 2.95) & (Dimu.mass < 3.25)]
        output['cutflow']['Quarkonia mass'] += ak.sum(ak.num(Dimu))

        # Get the Muons from Dimu, for cuts in their params
        Muon = ak.zip({'0': Muons[Dimu.t1muIdx], '1': Muons[Dimu.t2muIdx]})
        
        ## Rec muon cuts
        
        # Muon softId cuts
        soft_id = (Muon.slot0.softId > 0) & (Muon.slot1.softId > 0)
        Dimu = Dimu[soft_id]
        Muon = Muon[soft_id]

        # Muon pt and eta cuts
        muon_pt_cut = (Muon.slot0.pt > 3) & (Muon.slot1.pt > 3)
        Dimu = Dimu[muon_pt_cut]
        Muon = Muon[muon_pt_cut]
         
        output['cutflow']['Dimu muon pt cut'] += ak.sum(ak.num(Dimu))
        
        # Dimuon quantity
        #output['n_dimu'] += ak.sum(ak.num(Dimu))
        # Muon quantity
        #output['n_mu'] += ak.sum(ak.num(Muon.slot0))
        #output['n_mu'] += ak.sum(ak.num(Muon.slot1))
        
        #### Pilep correction
        evaluator = extract_mc_weight(pileup_file)
        corrections = evaluator['weight_histogram'](nPVtx)
        correcto_muons = np.repeat(corrections, ak.num(Dimu))
        
        
        # Dimu plots
        output['dimu_pt'].fill(pt=ak.flatten(Dimu.pt), weight=correcto_muons)
        output['dimu_eta'].fill(eta=ak.flatten(Dimu.eta), weight=correcto_muons)
        output['dimu_phi'].fill(phi=ak.flatten(Dimu.phi), weight=correcto_muons)
        output['dimu_mass'].fill(mass=ak.flatten(Dimu.mass), weight=correcto_muons)
        
        # Muon plots
        output['muon_pt'].fill(pt=ak.flatten(Muon.slot0.pt), weight=correcto_muons)
        output['muon_eta'].fill(eta=ak.flatten(Muon.slot0.eta), weight=correcto_muons)
        output['muon_phi'].fill(phi=ak.flatten(Muon.slot0.phi), weight=correcto_muons)
        output['muon_pt'].fill(pt=ak.flatten(Muon.slot1.pt), weight=correcto_muons)
        output['muon_eta'].fill(eta=ak.flatten(Muon.slot1.eta), weight=correcto_muons)
        output['muon_phi'].fill(phi=ak.flatten(Muon.slot1.phi), weight=correcto_muons)
        
        ## Dstar
        
        # Cuts for Dstar
        Dstar = Dstars[~Dstars.hasMuon]
        output['cutflow']['Dstar trk muon cut'] += ak.sum(ak.num(Dstar))

        Dstar = Dstar[(Dstar.Kpt > 0.5) & (Dstar.pipt > 0.5)]
        output['cutflow']['Dstar trk pt cut'] += ak.sum(ak.num(Dstar))

        Dstar = Dstar[(Dstar.Kchindof < 2.5) & (Dstar.pichindof < 2.5)]
        output['cutflow']['Dstar trk pt cut'] += ak.sum(ak.num(Dstar))

        Dstar = Dstar[(Dstar.KnValid > 4) & (Dstar.pinValid > 4) & (Dstar.KnPix > 1) & (Dstar.pinPix > 1)]
        output['cutflow']['Dstar trk hits cut'] += ak.sum(ak.num(Dstar))

        Dstar = Dstar[(Dstar.Kdxy < 0.1) & (Dstar.pidxy < 0.1)]
        output['cutflow']['Dstar trk pt cut'] += ak.sum(ak.num(Dstar))

        Dstar = Dstar[(Dstar.Kdz < 1) & (Dstar.pidz < 1)]
        output['cutflow']['Dstar trk pt cut'] += ak.sum(ak.num(Dstar))

        # pis cuts
        Dstar = Dstar[Dstar.pispt > 0.3]
        output['cutflow']['Dstar pis pt cut'] += ak.sum(ak.num(Dstar))

        Dstar = Dstar[Dstar.pischindof < 3]
        output['cutflow']['Dstar pis chi2 cut'] += ak.sum(ak.num(Dstar))

        Dstar = Dstar[Dstar.pisnValid > 2]
        output['cutflow']['Dstar pis hits cut'] += ak.sum(ak.num(Dstar))

        # D0 of Dstar cuts
        Dstar = Dstar[Dstar.D0cosphi > 0.99]
        output['cutflow']['Dstar D0 cosphi cut'] += ak.sum(ak.num(Dstar))

        Dstar = Dstar[(Dstar.D0mass < D0_PDG_MASS + 0.025) & (Dstar.D0mass > D0_PDG_MASS - 0.025)]
        output['cutflow']['Dstar D0 mass cut'] += ak.sum(ak.num(Dstar))

        Dstar = Dstar[Dstar.D0pt > 3]
        output['cutflow']['Dstar D0 pt cut'] += ak.sum(ak.num(Dstar))

        Dstar = Dstar[Dstar.D0dlSig > 3]
        output['cutflow']['Dstar D0 dlSig cut'] += ak.sum(ak.num(Dstar))

        Dstar['wrg_chg'] = (Dstar.Kchg == Dstar.pichg)
        
        # Dstar match
        #dstar_gen = dstar_match(Dstar[~Dstar.wrg_chg], GenPart)
        
        #gen_dstar = GenPart[np.absolute(GenPart.pdgId == 413)]  
        #gen_d0dstar = GenPart[np.absolute(GenPart.pdgId == 420)]  
        
        # Cuts for gendstar and gend0
        #gen_dstar = gen_dstar[gen_dstar.pt > 3]
        #gen_dstar = gen_dstar[(gen_dstar.eta <= 2.4) & (gen_dstar.eta >= -2.4)]
        #gen_d0dstar = gen_d0dstar[gen_d0dstar.pt > 3]
        
        correcto_dstar = np.repeat(corrections, ak.num(Dstar))
        
        # Dstar quantity
        output['n_dstar'] += ak.sum(ak.num(Dstar))
        
        # Histograms
        output['dstar_pt'].fill(pt=ak.flatten(Dstar.pt), weight=correcto_dstar)
        output['dstar_eta'].fill(eta=ak.flatten(Dstar.eta), weight=correcto_dstar)
        output['dstar_phi'].fill(phi=ak.flatten(Dstar.phi), weight=correcto_dstar)
        output['dstar_deltamr'].fill(deltamr=ak.flatten(Dstar.deltamr), weight=correcto_dstar)
        
    
        '''if len(DimuDstar) > 0:
            for i0 in range(len(DimuDstar)):
                print(i0)
                for i1 in range(len(DimuDstar[i0])):
                    print(f"Reco Dimu: ({DimuDstar.slot0.slot0.pt[i0][i1]}, {DimuDstar.slot0.slot0.eta[i0][i1]}, {DimuDstar.slot0.slot0.phi[i0][i1]})")
                    print(f" Gen Dimu: ({DimuDstar.slot0.slot1.pt[i0][i1]}, {DimuDstar.slot0.slot1.eta[i0][i1]}, {DimuDstar.slot0.slot1.phi[i0][i1]})")
                    print(f"Reco Dstar: ({DimuDstar.slot1.slot0.pt[i0][i1]}, {DimuDstar.slot1.slot0.eta[i0][i1]}, {DimuDstar.slot1.slot0.phi[i0][i1]})")
                    print(f" Gen Dstar: ({DimuDstar.slot1.slot1.pt[i0][i1]}, {DimuDstar.slot1.slot1.eta[i0][i1]}, {DimuDstar.slot1.slot1.phi[i0][i1]})")
                    print(f"Reco deltarap: {DimuDstar.deltarap[i0][i1]}, Gen deltarap: {DimuDstar.gen_deltarap[i0][i1]}")
                    print("-----------------------------------------------------------------")'''
        
      
        return output

    def postprocess(self, accumulator):
        return accumulator

In [49]:
data = {"test": files[:20]}

tstart = time.time()

output = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=ReweightTestProcessor(),
                                    executor=processor.iterative_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output['cutflow'])

Preprocessing:   0%|          | 0/19 [00:00<?, ?file/s]

Processing:   0%|          | 0/20 [00:00<?, ?chunk/s]

Process finished in: 72.85 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 2650, 'Number of Dimu': 3655, 'Number of Muons': 14917, 'Dimu 0 charge': 2524, 'Quarkonia mass': 1243, 'Dimu muon pt cut': 397, 'Dstar trk muon cut': 702743, 'Dstar trk pt cut': 2176031, 'Dstar trk hits cut': 463497, 'Dstar pis pt cut': 55641, 'Dstar pis chi2 cut': 53687, 'Dstar pis hits cut': 53687, 'Dstar D0 cosphi cut': 4901, 'Dstar D0 mass cut': 1082, 'Dstar D0 pt cut': 757, 'Dstar D0 dlSig cut': 130})


In [ ]:
create_plot1d(output['dimu_mass'])

In [ ]:
create_plot1d(output['dimu_pt'])

In [ ]:
create_plot1d(output['dimu_eta'])

In [ ]:
create_plot1d(output['dimu_phi'])

In [ ]:
create_plot1d(output['muon_pt'])

In [ ]:
create_plot1d(output['muon_eta'])

In [ ]:
create_plot1d(output['muon_phi'])

In [ ]:
create_plot1d(output['dstar_dmr'])

In [ ]:
create_plot1d(output['dimu_mass'])

In [ ]:
create_plot1d(output['dimu_mass'])

In [ ]:
create_plot1d(output['dimu_mass'])

In [14]:
################################# Tests #################################

import uproot

f = uproot.open("/eos/user/m/mabarros/Monte_Carlo/2017/0000/JpsiToMuMuwithDstar_MC_2017_NanoAODplus_100.root")

events = f['Events'].arrays([*gen_part_cols, 'nPVtx'])

ext = extractor()

ext.add_weight_sets(["weight_histogram weight_histogram pileup_weight.root"])
ext.finalize()

evaluator = ext.make_evaluator()

print("available evaluator keys:")
for key in evaluator.keys():
    print("\t", key)
print(f"key:", evaluator[key])
print(f"type of key:", type(evaluator[key]))

print(events.nPVtx)
corrections = evaluator['weight_histogram'](events.nPVtx)
print(corrections)

available evaluator keys:
	 weight_histogram
key: 1 dimensional histogram with axes:
	1: (array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90.,
       91., 92., 93., 94., 95., 96., 97., 98., 99.]),)

type of key: <class 'coffea.lookup_tools.dense_lookup.dense_lookup'>
[19, 24, 22, 19, 31, 24, 13, 27, 31, 20, ... 18, 17, 34, 37, 33, 35, 31, 33, 39, 24]
[0.936, 0.978, 0.981, 0.936, 1.02, 0.978, ... 1.04, 1.06, 1.02, 1.04, 1.06, 0.978]
